## 1. Environment Setup

In [ ]:
print("SERVICE HEALTH CHECK")
print("=" * 40)

services = {
    "FastAPI": "http://localhost:8000/api/v1/health",
    "OpenSearch": "http://localhost:9200/_cluster/health",
    "Ollama": "http://localhost:11434/api/version",
    "LangFuse": "http://localhost:3000/api/public/health"
}

all_healthy = True
for service_name, url in services.items():
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✓ {service_name}: Healthy")
        else:
            print(f"✗ {service_name}: HTTP {response.status_code}")
            all_healthy = False
    except Exception as e:
        print(f"✗ {service_name}: Not accessible - {e}")
        all_healthy = False

# Check Redis through API or directly
print("\nChecking Redis:")
try:
    # First try via API health endpoint
    response = requests.get("http://localhost:8000/api/v1/health")
    if response.status_code == 200:
        health_data = response.json()
        redis_info = health_data.get('services', {}).get('redis')
        if redis_info:
            redis_status = redis_info.get('status')
            if redis_status == 'healthy':
                print(f"✓ Redis: Healthy (via API)")
            else:
                print(f"✗ Redis: {redis_status or 'Unknown'}")
                all_healthy = False
        else:
            # Redis not in health endpoint, try direct connection
            print("ℹ Redis: Not in health endpoint, checking direct connection...")
            
            # Try to import redis and test connection
            try:
                import redis
                r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)
                r.ping()
                print("✓ Redis: Healthy (direct connection)")
            except ImportError:
                print("ℹ Redis: Python client not available in notebook environment")
                print("ℹ Redis: Assuming healthy (container running)")
            except Exception as redis_error:
                print(f"✗ Redis: Connection failed - {redis_error}")
                all_healthy = False
    else:
        print("✗ Cannot check Redis - API not responding")
        all_healthy = False
except Exception as e:
    print(f"✗ Redis: Could not check status - {e}")
    all_healthy = False

if all_healthy:
    print("\n✓ All services ready!")
else:
    print("\n⚠ Some services need attention. Run: docker compose up --build -d")

In [19]:
# Check API Endpoints
print("API STRUCTURE")
print("=" * 20)

try:
    response = requests.get("http://localhost:8000/openapi.json")
    if response.status_code == 200:
        openapi_data = response.json()
        endpoints = list(openapi_data['paths'].keys())
        
        print(f"Total endpoints: {len(endpoints)}")
        print("\nAvailable endpoints:")
        for endpoint in sorted(endpoints):
            print(f"  • {endpoint}")
      
    else:
        print(f"Could not fetch API info: {response.status_code}")
except Exception as e:
    print(f"Error: {e}")

API STRUCTURE
Total endpoints: 4

Available endpoints:
  • /api/v1/ask
  • /api/v1/health
  • /api/v1/hybrid-search/
  • /api/v1/stream


In [20]:
# Check Cache Status
print("CACHE CONFIGURATION")
print("=" * 40)

try:
    # Get health status 
    response = requests.get("http://localhost:8000/api/v1/health")
    if response.status_code == 200:
        health_data = response.json()
        print(f"API Status: {health_data.get('status', 'unknown')}")
        print(f"Cache Integration: Built into RAG endpoints")
        print(f"Cache Type: Redis")
        print(f"Cache Strategy: Exact parameter matching")
        print(f"TTL: Configurable (default 24 hours)")
        
        print(f"\n✓ Cache system is integrated and ready")
    else:
        print("Could not fetch API status")
except Exception as e:
    print(f"Error checking cache: {e}")

print(f"\nℹ️ Cache Testing Strategy:")
print(f"  1. First query: Full RAG pipeline (cache miss)")
print(f"  2. Identical query: Cached response (cache hit)")  
print(f"  3. Different query: Full RAG pipeline (cache miss)")

CACHE CONFIGURATION
API Status: ok
Cache Integration: Built into RAG endpoints
Cache Type: Redis
Cache Strategy: Exact parameter matching
TTL: Configurable (default 24 hours)

✓ Cache system is integrated and ready

ℹ️ Cache Testing Strategy:
  1. First query: Full RAG pipeline (cache miss)
  2. Identical query: Cached response (cache hit)
  3. Different query: Full RAG pipeline (cache miss)


## 3. API Structure Overview



In [21]:
# First Query - Should NOT use cache
print("FIRST QUERY TEST (NO CACHE - BASELINE)")
print("=" * 50)

test_query = "What are the latest advances in transformer models for NLP?"
print(f"Query: {test_query}")
print(f"\nExpected: Full RAG pipeline execution (15-20 seconds)")
print("-" * 50)

start_time = time.time()

try:
    request_data = {
        "query": test_query,
        "top_k": 3,
        "use_hybrid": True,
        "model": "llama3.2:1b"
    }
    
    print("\nSending request...")
    response = requests.post(
        "http://localhost:8000/api/v1/ask",
        json=request_data,
        timeout=60
    )
    
    first_query_time = time.time() - start_time
    
    if response.status_code == 200:
        data = response.json()
        
        print(f"\n✓ Success!")
        print(f"Response Time: {first_query_time:.2f} seconds")
        
        print(f"\nAnswer Preview:")
        print("-" * 50)
        answer_preview = data['answer'][:400] if len(data['answer']) > 400 else data['answer']
        print(answer_preview + ("..." if len(data['answer']) > 400 else ""))
        print("-" * 50)
        
        print(f"\nMetadata:")
        print(f"  • Sources: {len(data.get('sources', []))} papers")
        print(f"  • Chunks used: {data.get('chunks_used', 0)}")
        print(f"  • Search mode: {data.get('search_mode', 'hybrid')}")
        
        # Store for comparison
        first_answer = data['answer']
        first_response_data = data
        
    else:
        print(f"\n✗ Request failed: {response.status_code}")
        print(f"Response: {response.text[:200]}")
        first_query_time = None
        
except Exception as e:
    print(f"\n✗ Error: {e}")
    first_query_time = None

if first_query_time:
    print(f"\n📊 Baseline established: {first_query_time:.2f} seconds")

FIRST QUERY TEST (NO CACHE - BASELINE)
Query: What are the latest advances in transformer models for NLP?

Expected: Full RAG pipeline execution (15-20 seconds)
--------------------------------------------------

Sending request...

✓ Success!
Response Time: 0.24 seconds

Answer Preview:
--------------------------------------------------
Transformer models have made tremendous progress in recent years, with significant advancements in language understanding and generation. One area of focus is the development of more efficient quantization techniques to improve model deployment on consumer hardware. The latest research highlights the importance of learning-based orthogonal butterfly transforms (ButterflyQuant) for ultra-low-bit la...
--------------------------------------------------

Metadata:
  • Sources: 2 papers
  • Chunks used: 3
  • Search mode: hybrid

📊 Baseline established: 0.24 seconds


In [22]:
# Second Query - Should USE cache
print("SECOND QUERY TEST (WITH CACHE - OPTIMIZED)")
print("=" * 50)

# Same query as before
print(f"Query: {test_query}")
print(f"\nExpected: Cache hit (sub-second response)")
print("-" * 50)

# Small delay to ensure cache is written
time.sleep(0.5)

start_time = time.time()

try:
    request_data = {
        "query": test_query,
        "top_k": 3,
        "use_hybrid": True,
        "model": "llama3.2:1b"
    }
    
    print("\nSending identical request...")
    response = requests.post(
        "http://localhost:8000/api/v1/ask",
        json=request_data,
        timeout=60
    )
    
    second_query_time = time.time() - start_time
    
    if response.status_code == 200:
        data = response.json()
        
        print(f"\n✓ Success!")
        print(f"Response Time: {second_query_time:.3f} seconds ({second_query_time*1000:.0f}ms)")
        
        print(f"\nAnswer Preview:")
        print("-" * 50)
        answer_preview = data['answer'][:400] if len(data['answer']) > 400 else data['answer']
        print(answer_preview + ("..." if len(data['answer']) > 400 else ""))
        print("-" * 50)
        
        # Store for comparison
        second_answer = data['answer']
        
        # Performance comparison
        if first_query_time and second_query_time:
            speedup = first_query_time / second_query_time
            time_saved = first_query_time - second_query_time
            
            print(f"\n📊 PERFORMANCE COMPARISON")
            print("=" * 50)
            print(f"First Query (no cache): {first_query_time:.2f} seconds")
            print(f"Second Query (cached): {second_query_time:.3f} seconds")
            print(f"\n🚀 Speed Improvement: {speedup:.0f}x faster")
            print(f"⏱️ Time Saved: {time_saved:.2f} seconds")
            
            # Verify answers are identical
            if first_answer == second_answer:
                print(f"\n✓ Answers are identical (cache working correctly)")
            else:
                print(f"\n⚠ Answers differ (cache may not be active)")
            
            if speedup > 50:
                print(f"\n🎉 Achieved {speedup:.0f}x performance improvement!")
                print(f"   This demonstrates production-grade caching!")
        
    else:
        print(f"\n✗ Request failed: {response.status_code}")
        second_query_time = None
        
except Exception as e:
    print(f"\n✗ Error: {e}")
    second_query_time = None

SECOND QUERY TEST (WITH CACHE - OPTIMIZED)
Query: What are the latest advances in transformer models for NLP?

Expected: Cache hit (sub-second response)
--------------------------------------------------

Sending identical request...

✓ Success!
Response Time: 0.131 seconds (131ms)

Answer Preview:
--------------------------------------------------
Transformer models have made tremendous progress in recent years, with significant advancements in language understanding and generation. One area of focus is the development of more efficient quantization techniques to improve model deployment on consumer hardware. The latest research highlights the importance of learning-based orthogonal butterfly transforms (ButterflyQuant) for ultra-low-bit la...
--------------------------------------------------

📊 PERFORMANCE COMPARISON
First Query (no cache): 0.24 seconds
Second Query (cached): 0.131 seconds

🚀 Speed Improvement: 2x faster
⏱️ Time Saved: 0.10 seconds

✓ Answers are identical (cache wo